In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import journal_figure
from scipy.stats import pearsonr, spearmanr

method_name = "NoiseTunnel_Saliency"
model_name = "convnext"
path_template = f"./attribution_maps/{model_name}/{method_name}/image_{{i}}.npy"
epsilon = 1e-8

n_images = 50  # or len(test_dataset) if available

In [ ]:
def normalize_map(arr):
    """Normalize attribution map for fair comparison."""
    arr = arr.astype(np.float32)
    arr -= arr.mean()
    arr /= (arr.std() + 1e-8)
    return arr

In [ ]:
l1_sims = []
l2_sims = []

for i in range(n_images):
    map_i = normalize_map(np.load(path_template.format(i=i)))
    for j in range(i + 1, n_images):
        map_j = normalize_map(np.load(path_template.format(i=j)))

        diff = map_i - map_j

        l1_dist = np.sum(np.abs(diff))
        l2_dist = np.sqrt(np.sum(diff ** 2))

        l1_sim = 1.0 / (l1_dist + epsilon)
        l2_sim = 1.0 / (l2_dist + epsilon)

        l1_sims.append(l1_sim)
        l2_sims.append(l2_sim)

l1_sims = np.array(l1_sims)
l2_sims = np.array(l2_sims)

plt.figure(figsize=(6, 6))
plt.scatter(l1_sims, l2_sims, s=20, color="royalblue", alpha=0.6, edgecolor="none")
plt.xlabel("L1 similarity")
plt.ylabel("L2 similarity")
journal_figure()


In [ ]:
import torch
import lpips
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lpips_model = lpips.LPIPS(net='alex').to(device)

In [ ]:
def prep_for_lpips(arr):
    """Prepare tensor for LPIPS: 3xHxW in [-1,1]."""
    arr = normalize_map(arr)
    if arr.ndim == 2:
        arr = np.stack([arr]*3, axis=0)  # make 3 channels
    elif arr.shape[0] == 1:
        arr = np.repeat(arr, 3, axis=0)
    arr = np.clip(arr / (np.max(np.abs(arr)) + 1e-8), -1, 1)
    return torch.from_numpy(arr).unsqueeze(0).to(device)

In [ ]:
l1_dists, l2_dists, lpips_dists = [], [], []

for i in range(n_images):
    map_i = normalize_map(np.load(path_template.format(i=i)))
    for j in range(i+1, n_images):
        map_j = normalize_map(np.load(path_template.format(i=j)))

        diff = map_i - map_j

        l1_dists.append(1.0 / (np.sum(np.abs(diff)) + epsilon))
        l2_dists.append(1.0 / (np.sqrt(np.sum(diff**2)) + epsilon))

        xi, xj = prep_for_lpips(map_i), prep_for_lpips(map_j)
        val = lpips_model(xi, xj).item()
        lpips_dists.append(1.0 / (val + epsilon))

l1_dists, l2_dists, lpips_dists = map(np.array, [l1_dists, l2_dists, lpips_dists])

print(f"Pearson(LPIPS, L1): {pearsonr(lpips_dists, l1_dists)[0]:.3f}")
print(f"Pearson(LPIPS, L2): {pearsonr(lpips_dists, l2_dists)[0]:.3f}")

print(f"Spearman(LPIPS, L1): {spearmanr(lpips_dists, l1_dists)}")
print(f"Spearman(LPIPS, L2): {spearmanr(lpips_dists, l2_dists)}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(lpips_dists, l1_dists, s=20, alpha=0.6, color='darkorange')
plt.xlabel("LPIPS similarity")
plt.ylabel("L1 similarity")
plt.title("LPIPS vs L1")
journal_figure()

plt.subplot(1, 2, 2)
plt.scatter(lpips_dists, l2_dists, s=20, alpha=0.6, color='royalblue')
plt.xlabel("LPIPS similarity")
plt.ylabel("L2 similarity")
plt.title("LPIPS vs L2")
journal_figure()